In [42]:
import pandas as pd


df = pd.read_excel('stokas.xlsx')

df.head()

,clean,label
0,gaji guru naik hingga juta p al mendukung prog...,p
1,lihat ganjar di makassar kemarin terutama vide...,p
2,ganjar mahfud targetkan satu desa satu puskesm...,p
3,calon presiden capres nomor urut anies basweda...,p
4,saat pagi tadi pak anies ada di tanah merah ko...,p


In [43]:
import re
import emoji
import string
import pandas as pd

def preprocess_text(original_text):
    # Menghapus URL
    processed_text = re.sub(r'http\S+', '', original_text)

    # Menghapus username Twitter (@user123)
    processed_text = re.sub(r'@\w+', '', processed_text)

    # Menghapus hashtag (#PythonProgramming)
    processed_text = re.sub(r'#\w+', '', processed_text)

    # Menghapus karakter yang berulang, lebih dari 2 kali
    processed_text = re.sub(r'(\w)\1{2,}', r'\1', processed_text)

    # Membersihkan emoji tanpa menghapus
    emojis = set([char for word in processed_text.split() for char in list(word) if char in emoji.UNICODE_EMOJI])
    emojis_names = [emoji.demojize(e).replace(':', '').replace('_', ' ') for e in emojis]
    processed_text = processed_text + ' ' + ' '.join(emojis_names)

    # Menghapus tanda baca
    processed_text = processed_text.translate(str.maketrans('', '', string.punctuation))

    # Menghapus angka
    processed_text = re.sub(r'\d+', '', processed_text)

    return processed_text.strip().lower()

# Applying the preprocess_text function to the 'full_text' column
df['clean'] = df['clean'].apply(lambda x: preprocess_text(x))


In [44]:
indo_slang_word = pd.read_csv("https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv")
slang_to_formal = dict(zip(indo_slang_word['slang'], indo_slang_word['formal']))

# Fungsi normalisasi teks
def normalize_text(text):
    words = text.split()  # Memecah teks menjadi kata-kata
    normalized_words = []
    for word in words:
        if word in slang_to_formal:
            normalized_words.append(slang_to_formal[word])
        else:
            normalized_words.append(word)
    normalized_text = ' '.join(normalized_words)  # Menggabungkan kata-kata kembali menjadi teks
    return normalized_text

df['clean'] = df['clean'].apply(normalize_text)

In [45]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(tweet):
    text = [stemmer.stem(word) for word in tweet]
    return tweet

df['clean'] = df['clean'].apply(lambda x: stemming(x))

In [46]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords = stopwords.words('indonesian')
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt','lg','min','kak','kah','no','deh','kayak','ky','woy','gue','hai','halo',
                       '&amp', 'yah','an','my','ane','btw','lu',"b", "e",'blok','a','duh','apa','om','lho','dear','minn','hi','iki','zimi','fo','iya','guys','l','tks',
                       'tbtb','yuk','pas','hm','heh','fax','lo','cuk','mas','eh'])

# Baca daftar stopword dari URL
stopwords_url = 'https://raw.githubusercontent.com/masdevid/ID-Stopwords/master/id.stopwords.02.01.2016.txt'
stopwords_df = pd.read_csv(stopwords_url, header=None, names=['stopword'])
stopwords_list = stopwords_df['stopword'].tolist()

def remove_stopwords(text):
    words = word_tokenize(str(text))
    filtered_words = [word for word in words if word.lower() not in list_stopwords]
    return ' '.join(filtered_words)

df['clean'] = df['clean'].apply(remove_stopwords)


In [47]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import pandas as pd

# Membuat instance dari StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword_remover = factory.create_stop_word_remover()


# Menerapkan penghapusan stopword pada kolom 'clean'
df['clean'] = df['clean'].apply(stopword_remover.remove)



In [48]:
df

,clean,label
0,gaji guru juta p al mendukung program ganjarma...,p
1,lihat ganjar makassar kemarin video membuktika...,p
2,ganjar mahfud targetkan desa puskesmas dokter ...,p
3,calon presiden capres nomor urut anies basweda...,p
4,pagi anies tanah merah koja jakarta utara anie...,p
5,gen menyesal pilih pasangan amin tesis anies b...,p
6,prestasi gibran kota solo dinikmati rakyat ind...,p
7,drama gara gara gibran bu iriana nama indonesi...,p
8,kampanye tkn prabowogibran langsung luncurkan ...,p
9,organisasi didirikan wapres arus indonesia arb...,p


In [50]:
positif = df[df['label'] == 'p']
positif.shape

(14, 2)

In [51]:
negatif = df[df['label'] == 'n']
negatif.shape

(15, 2)

In [41]:
positif.to_csv('positif.csv')
# negatif.to_csv('negatif.csv')

In [ ]:
df.to_csv('result.csv')